## Solving bidomain equations on rectangular domain with Beeler-Reuter model

### Importing required libraries

In [1]:
%cd ..
from src.dynamics_models import *
from src.cell_models import *
from src.utils import *

/home/drvinko/Desktop/Modeliranje-biofizike-srcanog-misica


### Defining a domain, cell model and dynamics model and visualizing initial data

In [2]:
domain = rectangle(1, 1, 30, 30)
reduce_factor = 5

In [3]:
class Model(MonodomainModel):
    def initial_V_m(self):
        def value(x):
            mu, alpha = 0.1, 30
            return -85 + 35 / (1 + np.exp((x[0] - mu) * alpha))

        self.V_m_n.interpolate(value)
        return self.V_m_n

    def conductivity(self):
        # Muscle fibres
        self.fibers = ufl.as_vector([1, 0])

        # Healthy conductivity
        self.M_i = self.SIGMA_IT * ufl.Identity(2) + (
            self.SIGMA_IL - self.SIGMA_IT
        ) * ufl.outer(self.fibers, self.fibers)

    def ischemia(self):
        x_c, width = 0.5, 0.25
        sharpness = 80.0

        def reduce(x):
            return 1 + (reduce_factor - 1) / (
                1 + ufl.cosh(sharpness * (x[0] - x_c)) / ufl.cosh(sharpness * width)
            ) / (1 + ufl.cosh(sharpness * (x[1] - x_c)) / ufl.cosh(sharpness * width))

        def value(x):
            return 1 + (reduce_factor - 1) / (
                1 + np.cosh(sharpness * (x[0] - x_c)) / np.cosh(sharpness * width)
            ) / (1 + np.cosh(sharpness * (x[1] - x_c)) / np.cosh(sharpness * width))

        self.M_i = self.M_i / reduce(self.x)
        fun = fem.Function(self.V1)
        fun.interpolate(value)
        return fun

In [4]:
cell_model = BeelerReuter(domain)
model = Model(domain, cell_model)

In [5]:
plot_function(
    model.initial_V_m(),
    camera_direction="xy",
    zoom=0.9,
    cmap='PiYG',
    save_to="rectangle_initial_V_m.pdf",
)

Widget(value='<iframe src="http://localhost:44113/index.html?ui=P_0x7ff2ce534110_0&reconnect=auto" class="pyvi…

In [6]:
plot_function(
    model.ischemia(),
    camera_direction="xy",
    cmap="RdYlGn",
    save_to="rectangle_ischemia_location.pdf",
)

Widget(value='<iframe src="http://localhost:44113/index.html?ui=P_0x7ff28981bc10_1&reconnect=auto" class="pyvi…

### Solving equations with given parameters

In [7]:
model.solve(
    T=300,
    steps=8000,
    save_to=f"rectangle_BR_{reduce_factor}_red.mp4",
    camera_direction="xy",
    zoom=0.9,
    checkpoints=[int(300 * i / 10) for i in range(1, 11)],
    checkpoint_file=f"rectangle_BR_{reduce_factor}_red",
)

Solving problem: 100%|██████████| 8000/8000 [09:34<00:00, 13.92it/s]


### Plotting fiber orientations

In [8]:
#plot_vector_field(
#    domain,
#    lambda x: [
#        5 * x[1] ** 2 / np.sqrt((25 * x[1] ** 4 + 1)),
#        1 / np.sqrt((25 * x[1] ** 4 + 1)),
#        0,
#    ],
#    0.03,
#    0.1,
#    camera_direction="xy",
#    save_to="rectangle_fibers.pdf",
#)